# Business Intelligence and Executive Dashboards

This notebook demonstrates business intelligence capabilities in the Enterprise Data Analysis Cognitive Architecture. We'll create executive dashboards, KPI tracking, and automated reporting systems.

## What You'll Learn
- Executive dashboard design principles
- KPI development and tracking
- Interactive data visualizations
- Automated report generation
- Business intelligence best practices
- Stakeholder communication strategies

## Prerequisites
- Completion of 01_getting_started.ipynb
- Basic understanding of business metrics
- Familiarity with data visualization concepts

## 1. Setup and Business Context

In [5]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Enterprise components
import sys
sys.path.append('../src')
try:
    from data_loader import DataLoader
    from statistical_analyzer import StatisticalAnalyzer
    from visualizer import EnterpriseVisualizer
    print("✅ Enterprise components loaded successfully")
except ImportError:
    print("⚠️ Enterprise components not found - using standard libraries only")

# Configuration
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")
np.random.seed(42)

# Set up for interactive plots
pyo.init_notebook_mode(connected=True)

print("📊 Business Intelligence environment ready!")

✅ Enterprise components loaded successfully


📊 Business Intelligence environment ready!


## 2. Business Data Generation and Preparation

In [2]:
# Create comprehensive business dataset for BI
print("🏢 CREATING BUSINESS INTELLIGENCE DATASET")
print("=" * 55)

# Generate business performance data over time
date_range = pd.date_range(start='2023-01-01', end='2025-07-19', freq='D')
n_days = len(date_range)

# Base business metrics with seasonality and trends
np.random.seed(42)

# Revenue data with growth trend and seasonality
base_revenue = 100000
growth_rate = 0.0003  # Daily growth
seasonal_factor = 0.2
weekend_factor = 0.7

daily_revenue = []
for i, date in enumerate(date_range):
    # Base growth trend
    trend_revenue = base_revenue * (1 + growth_rate * i)
    
    # Seasonal adjustments (monthly cycle)
    seasonal_adj = 1 + seasonal_factor * np.sin(2 * np.pi * i / 30)
    
    # Weekend adjustment
    if date.weekday() >= 5:  # Saturday = 5, Sunday = 6
        weekend_adj = weekend_factor
    else:
        weekend_adj = 1.0
    
    # Random variation
    random_factor = np.random.normal(1, 0.15)
    
    revenue = trend_revenue * seasonal_adj * weekend_adj * random_factor
    daily_revenue.append(max(revenue, 0))

# Customer metrics
daily_customers = []
customer_base = 5000
for i, revenue in enumerate(daily_revenue):
    # Customers correlated with revenue but with own variation
    avg_transaction = np.random.normal(200, 50)
    customers = int(revenue / max(avg_transaction, 50))
    daily_customers.append(customers)

# Operational metrics
daily_orders = [int(customers * np.random.uniform(1.1, 1.8)) for customers in daily_customers]
daily_returns = [int(orders * np.random.uniform(0.02, 0.08)) for orders in daily_orders]
daily_support_tickets = [int(customers * np.random.uniform(0.05, 0.15)) for customers in daily_customers]

# Marketing metrics
daily_marketing_spend = [revenue * np.random.uniform(0.05, 0.12) for revenue in daily_revenue]
daily_leads = [int(spend / np.random.uniform(8, 15)) for spend in daily_marketing_spend]
daily_conversions = [int(leads * np.random.uniform(0.15, 0.35)) for leads in daily_leads]

# Employee metrics
base_employees = 150
daily_employees = [base_employees + int(i / 30) for i in range(n_days)]  # Growing workforce

# Product categories
categories = ['Electronics', 'Clothing', 'Home & Garden', 'Sports', 'Books']
category_weights = [0.35, 0.25, 0.2, 0.15, 0.05]

# Regional data
regions = ['North America', 'Europe', 'Asia Pacific', 'Latin America']
region_weights = [0.4, 0.3, 0.2, 0.1]

# Create comprehensive business dataset
business_data = {
    'date': date_range,
    'revenue': daily_revenue,
    'customers': daily_customers,
    'orders': daily_orders,
    'returns': daily_returns,
    'support_tickets': daily_support_tickets,
    'marketing_spend': daily_marketing_spend,
    'leads_generated': daily_leads,
    'conversions': daily_conversions,
    'employees': daily_employees
}

# Add derived metrics
business_data['avg_order_value'] = [r/o if o > 0 else 0 for r, o in zip(daily_revenue, daily_orders)]
business_data['return_rate'] = [ret/ord if ord > 0 else 0 for ret, ord in zip(daily_returns, daily_orders)]
business_data['conversion_rate'] = [conv/lead if lead > 0 else 0 for conv, lead in zip(daily_conversions, daily_leads)]
business_data['customer_acquisition_cost'] = [spend/conv if conv > 0 else 0 for spend, conv in zip(daily_marketing_spend, daily_conversions)]
business_data['revenue_per_employee'] = [r/e if e > 0 else 0 for r, e in zip(daily_revenue, daily_employees)]

# Create detailed transactional data for more granular analysis
transaction_data = []
transaction_id = 1

for i, date in enumerate(date_range):
    daily_order_count = daily_orders[i]
    daily_rev = daily_revenue[i]
    
    for order_idx in range(daily_order_count):
        # Distribute revenue across orders with variation
        base_order_value = daily_rev / daily_order_count
        order_value = max(base_order_value * np.random.normal(1, 0.5), 10)
        
        # Select random category and region
        category = np.random.choice(categories, p=category_weights)
        region = np.random.choice(regions, p=region_weights)
        
        # Customer type
        customer_type = np.random.choice(['New', 'Returning', 'VIP'], p=[0.3, 0.6, 0.1])
        
        transaction_data.append({
            'transaction_id': transaction_id,
            'date': date,
            'order_value': order_value,
            'category': category,
            'region': region,
            'customer_type': customer_type,
            'payment_method': np.random.choice(['Credit Card', 'PayPal', 'Bank Transfer'], p=[0.6, 0.25, 0.15]),
            'delivery_days': np.random.randint(1, 8)
        })
        
        transaction_id += 1
        
        # Limit to reasonable number of transactions for memory
        if len(transaction_data) > 100000:
            break
    
    if len(transaction_data) > 100000:
        break

# Create DataFrames
df_business = pd.DataFrame(business_data)
df_transactions = pd.DataFrame(transaction_data)

# Add time-based features
df_business['year'] = df_business['date'].dt.year
df_business['month'] = df_business['date'].dt.month
df_business['quarter'] = df_business['date'].dt.quarter
df_business['day_of_week'] = df_business['date'].dt.dayofweek
df_business['is_weekend'] = df_business['day_of_week'] >= 5

df_transactions['year'] = df_transactions['date'].dt.year
df_transactions['month'] = df_transactions['date'].dt.month
df_transactions['quarter'] = df_transactions['date'].dt.quarter

print(f"📈 Created business dataset:")
print(f"  • Daily metrics: {len(df_business):,} days ({df_business['date'].min()} to {df_business['date'].max()})")
print(f"  • Transactions: {len(df_transactions):,} records")
print(f"  • Revenue range: ${df_business['revenue'].min():,.0f} - ${df_business['revenue'].max():,.0f}")
print(f"  • Total revenue: ${df_business['revenue'].sum():,.0f}")

# Business KPIs summary
current_metrics = df_business.tail(30).mean()  # Last 30 days average
print(f"\n📊 Current KPIs (30-day average):")
print(f"  • Daily Revenue: ${current_metrics['revenue']:,.0f}")
print(f"  • Daily Customers: {current_metrics['customers']:,.0f}")
print(f"  • Average Order Value: ${current_metrics['avg_order_value']:,.0f}")
print(f"  • Conversion Rate: {current_metrics['conversion_rate']:.1%}")
print(f"  • Return Rate: {current_metrics['return_rate']:.1%}")

df_business.head()

🏢 CREATING BUSINESS INTELLIGENCE DATASET
📈 Created business dataset:
  • Daily metrics: 931 days (2023-01-01 00:00:00 to 2025-07-19 00:00:00)
  • Transactions: 100,001 records
  • Revenue range: $44,396 - $204,502
  • Total revenue: $97,312,773

📊 Current KPIs (30-day average):
  • Daily Revenue: $119,236
  • Daily Customers: 618
  • Average Order Value: $147
  • Conversion Rate: 24.3%
  • Return Rate: 5.0%
📈 Created business dataset:
  • Daily metrics: 931 days (2023-01-01 00:00:00 to 2025-07-19 00:00:00)
  • Transactions: 100,001 records
  • Revenue range: $44,396 - $204,502
  • Total revenue: $97,312,773

📊 Current KPIs (30-day average):
  • Daily Revenue: $119,236
  • Daily Customers: 618
  • Average Order Value: $147
  • Conversion Rate: 24.3%
  • Return Rate: 5.0%


,date,revenue,customers,orders,returns,support_tickets,marketing_spend,leads_generated,conversions,employees,avg_order_value,return_rate,conversion_rate,customer_acquisition_cost,revenue_per_employee,year,month,quarter,day_of_week,is_weekend
0,2023-01-01,75215.498607,255,337,20,19,4217.292370,449,126,150,223.191391,0.059347,0.280624,33.470574,501.436657,2023,1,1,6,True
1,2023-01-02,102028.628414,799,1204,59,49,8946.422843,612,187,150,84.741386,0.049003,0.305556,47.841833,680.190856,2023,1,1,0,False
2,2023-01-03,118711.561144,1318,1792,36,142,13524.342579,911,208,150,66.245291,0.020089,0.228321,65.020878,791.410408,2023,1,1,1,False
3,2023-01-04,137410.354474,618,939,64,90,10602.320883,997,278,150,146.336906,0.068158,0.278837,38.137845,916.069030,2023,1,1,2,False
4,2023-01-05,110961.560562,634,1010,67,55,11991.572375,829,149,150,109.862931,0.066337,0.179735,80.480352,739.743737,2023,1,1,3,False


## 3. Executive Dashboard - Key Performance Indicators

In [3]:
# Executive Dashboard - Key Performance Indicators
print("📊 EXECUTIVE DASHBOARD - KEY PERFORMANCE INDICATORS")
print("=" * 65)

# Calculate key business metrics
recent_data = df_business.tail(90)  # Last 90 days
previous_data = df_business.iloc[-180:-90]  # Previous 90 days for comparison

# KPI calculations
current_kpis = {
    'total_revenue': recent_data['revenue'].sum(),
    'avg_daily_revenue': recent_data['revenue'].mean(),
    'total_customers': recent_data['customers'].sum(),
    'avg_order_value': recent_data['avg_order_value'].mean(),
    'conversion_rate': recent_data['conversion_rate'].mean(),
    'return_rate': recent_data['return_rate'].mean(),
    'cac': recent_data['customer_acquisition_cost'].mean(),
    'revenue_per_employee': recent_data['revenue_per_employee'].mean()
}

previous_kpis = {
    'total_revenue': previous_data['revenue'].sum(),
    'avg_daily_revenue': previous_data['revenue'].mean(),
    'total_customers': previous_data['customers'].sum(),
    'avg_order_value': previous_data['avg_order_value'].mean(),
    'conversion_rate': previous_data['conversion_rate'].mean(),
    'return_rate': previous_data['return_rate'].mean(),
    'cac': previous_data['customer_acquisition_cost'].mean(),
    'revenue_per_employee': previous_data['revenue_per_employee'].mean()
}

# Calculate growth rates
growth_rates = {}
for kpi in current_kpis:
    if previous_kpis[kpi] != 0:
        growth_rates[kpi] = ((current_kpis[kpi] - previous_kpis[kpi]) / previous_kpis[kpi]) * 100
    else:
        growth_rates[kpi] = 0

print("📈 90-Day Performance Summary:")
print("-" * 35)
for kpi, current_val in current_kpis.items():
    growth = growth_rates[kpi]
    trend_icon = "📈" if growth > 0 else "📉" if growth < 0 else "➡️"
    
    if 'rate' in kpi or 'conversion' in kpi:
        print(f"{kpi.replace('_', ' ').title()}: {current_val:.1%} ({growth:+.1f}%) {trend_icon}")
    elif 'revenue' in kpi or 'cac' in kpi:
        print(f"{kpi.replace('_', ' ').title()}: ${current_val:,.0f} ({growth:+.1f}%) {trend_icon}")
    else:
        print(f"{kpi.replace('_', ' ').title()}: {current_val:,.0f} ({growth:+.1f}%) {trend_icon}")

# Create Executive Dashboard with Plotly
print("\n🎨 Creating Interactive Executive Dashboard...")

# Create subplots for dashboard
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Revenue Trend', 'Customer Metrics', 'Order Performance',
                   'Marketing Efficiency', 'Regional Performance', 'Growth Indicators',
                   'Operational Metrics', 'Financial Health', 'Key Ratios'),
    specs=[[{"secondary_y": False}, {"secondary_y": True}, {"secondary_y": False}],
           [{"secondary_y": False}, {"type": "pie"}, {"secondary_y": False}],
           [{"secondary_y": True}, {"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Revenue Trend (Row 1, Col 1)
monthly_revenue = df_business.groupby(['year', 'month'])['revenue'].sum().reset_index()
monthly_revenue['date'] = pd.to_datetime(monthly_revenue[['year', 'month']].assign(day=1))

fig.add_trace(
    go.Scatter(x=monthly_revenue['date'], y=monthly_revenue['revenue'],
              mode='lines+markers', name='Monthly Revenue',
              line=dict(color='#1f77b4', width=3)),
    row=1, col=1
)

# 2. Customer Metrics (Row 1, Col 2)
monthly_customers = df_business.groupby(['year', 'month'])['customers'].sum().reset_index()
monthly_customers['date'] = pd.to_datetime(monthly_customers[['year', 'month']].assign(day=1))

fig.add_trace(
    go.Scatter(x=monthly_customers['date'], y=monthly_customers['customers'],
              mode='lines+markers', name='Customers', line=dict(color='#ff7f0e')),
    row=1, col=2
)

monthly_orders = df_business.groupby(['year', 'month'])['orders'].sum().reset_index()
monthly_orders['date'] = pd.to_datetime(monthly_orders[['year', 'month']].assign(day=1))

fig.add_trace(
    go.Scatter(x=monthly_orders['date'], y=monthly_orders['orders'],
              mode='lines+markers', name='Orders', line=dict(color='#2ca02c')),
    row=1, col=2, secondary_y=True
)

# 3. Order Performance (Row 1, Col 3)
monthly_aov = df_business.groupby(['year', 'month'])['avg_order_value'].mean().reset_index()
monthly_aov['date'] = pd.to_datetime(monthly_aov[['year', 'month']].assign(day=1))

fig.add_trace(
    go.Scatter(x=monthly_aov['date'], y=monthly_aov['avg_order_value'],
              mode='lines+markers', name='Avg Order Value',
              line=dict(color='#d62728', width=2)),
    row=1, col=3
)

# 4. Marketing Efficiency (Row 2, Col 1)
monthly_marketing = df_business.groupby(['year', 'month']).agg({
    'marketing_spend': 'sum',
    'conversions': 'sum'
}).reset_index()
monthly_marketing['date'] = pd.to_datetime(monthly_marketing[['year', 'month']].assign(day=1))
monthly_marketing['roi'] = (monthly_revenue['revenue'] / monthly_marketing['marketing_spend'] - 1) * 100

fig.add_trace(
    go.Scatter(x=monthly_marketing['date'], y=monthly_marketing['roi'],
              mode='lines+markers', name='Marketing ROI (%)',
              line=dict(color='#9467bd', width=2)),
    row=2, col=1
)

# 5. Regional Performance (Row 2, Col 2) - Pie Chart
regional_revenue = df_transactions.groupby('region')['order_value'].sum()

fig.add_trace(
    go.Pie(labels=regional_revenue.index, values=regional_revenue.values,
           name="Regional Revenue", hole=0.3),
    row=2, col=2
)

# 6. Growth Indicators (Row 2, Col 3)
# Calculate month-over-month growth
monthly_data = df_business.groupby(['year', 'month']).agg({
    'revenue': 'sum',
    'customers': 'sum'
}).reset_index()

monthly_data['revenue_growth'] = monthly_data['revenue'].pct_change() * 100
monthly_data['customer_growth'] = monthly_data['customers'].pct_change() * 100
monthly_data['date'] = pd.to_datetime(monthly_data[['year', 'month']].assign(day=1))

fig.add_trace(
    go.Scatter(x=monthly_data['date'], y=monthly_data['revenue_growth'],
              mode='lines+markers', name='Revenue Growth %',
              line=dict(color='#17becf')),
    row=2, col=3
)

# 7. Operational Metrics (Row 3, Col 1)
monthly_returns = df_business.groupby(['year', 'month'])['return_rate'].mean().reset_index()
monthly_returns['date'] = pd.to_datetime(monthly_returns[['year', 'month']].assign(day=1))

fig.add_trace(
    go.Scatter(x=monthly_returns['date'], y=monthly_returns['return_rate']*100,
              mode='lines+markers', name='Return Rate %',
              line=dict(color='#e377c2')),
    row=3, col=1
)

monthly_support = df_business.groupby(['year', 'month'])['support_tickets'].sum().reset_index()
monthly_support['date'] = pd.to_datetime(monthly_support[['year', 'month']].assign(day=1))

fig.add_trace(
    go.Scatter(x=monthly_support['date'], y=monthly_support['support_tickets'],
              mode='lines+markers', name='Support Tickets',
              line=dict(color='#bcbd22')),
    row=3, col=1, secondary_y=True
)

# 8. Financial Health (Row 3, Col 2)
quarterly_data = df_business.groupby(['year', 'quarter']).agg({
    'revenue': 'sum',
    'marketing_spend': 'sum'
}).reset_index()

quarterly_data['profit_margin'] = ((quarterly_data['revenue'] - quarterly_data['marketing_spend']) / 
                                 quarterly_data['revenue']) * 100
quarterly_data['quarter_label'] = quarterly_data['year'].astype(str) + '-Q' + quarterly_data['quarter'].astype(str)

fig.add_trace(
    go.Bar(x=quarterly_data['quarter_label'], y=quarterly_data['profit_margin'],
           name='Profit Margin %', marker_color='#2ca02c'),
    row=3, col=2
)

# 9. Key Ratios (Row 3, Col 3)
monthly_cac = df_business.groupby(['year', 'month'])['customer_acquisition_cost'].mean().reset_index()
monthly_cac['date'] = pd.to_datetime(monthly_cac[['year', 'month']].assign(day=1))

fig.add_trace(
    go.Scatter(x=monthly_cac['date'], y=monthly_cac['customer_acquisition_cost'],
              mode='lines+markers', name='Customer Acquisition Cost',
              line=dict(color='#ff7f0e')),
    row=3, col=3
)

# Update layout
fig.update_layout(
    height=1200,
    showlegend=False,
    title_text="<b>Executive Dashboard - Business Performance Overview</b>",
    title_x=0.5,
    title_font_size=20
)

# Update axes labels
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Revenue ($)", row=1, col=1)

fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_yaxes(title_text="Customers", row=1, col=2)
fig.update_yaxes(title_text="Orders", row=1, col=2, secondary_y=True)

fig.update_xaxes(title_text="Date", row=1, col=3)
fig.update_yaxes(title_text="Average Order Value ($)", row=1, col=3)

# Show the dashboard
fig.show()

print("✅ Executive Dashboard created successfully!")
print("\n📋 Dashboard includes:")
print("  • Revenue trends and growth patterns")
print("  • Customer acquisition and retention metrics")
print("  • Order performance and value analysis")
print("  • Marketing ROI and efficiency tracking")
print("  • Regional performance breakdown")
print("  • Operational metrics and quality indicators")
print("  • Financial health and profitability metrics")

📊 EXECUTIVE DASHBOARD - KEY PERFORMANCE INDICATORS
📈 90-Day Performance Summary:
-----------------------------------
Total Revenue: $10,839,618 (+4.6%) 📈
Avg Daily Revenue: $120,440 (+4.6%) 📈
Total Customers: 60,123 (+12.7%) 📈
Avg Order Value: 138 (-7.0%) 📉
Conversion Rate: 24.3% (+3.2%) 📈
Return Rate: 4.9% (+3.9%) 📈
Cac: $50 (-2.8%) 📉
Revenue Per Employee: $673 (+2.9%) 📈

🎨 Creating Interactive Executive Dashboard...


✅ Executive Dashboard created successfully!

📋 Dashboard includes:
  • Revenue trends and growth patterns
  • Customer acquisition and retention metrics
  • Order performance and value analysis
  • Marketing ROI and efficiency tracking
  • Regional performance breakdown
  • Operational metrics and quality indicators
  • Financial health and profitability metrics


## 4. Automated Business Insights and Alerts

In [6]:
# Automated Business Insights and Alert System
print("🤖 AUTOMATED BUSINESS INSIGHTS AND ALERTS")
print("=" * 55)

# Business Intelligence Alert System
class BusinessIntelligenceAlert:
    def __init__(self, data):
        self.data = data
        self.alerts = []
        self.insights = []
    
    def detect_anomalies(self):
        """Detect unusual patterns in business metrics"""
        recent_7_days = self.data.tail(7)
        previous_7_days = self.data.iloc[-14:-7]
        
        # Revenue anomaly detection
        recent_avg_revenue = recent_7_days['revenue'].mean()
        previous_avg_revenue = previous_7_days['revenue'].mean()
        revenue_change = ((recent_avg_revenue - previous_avg_revenue) / previous_avg_revenue) * 100
        
        if abs(revenue_change) > 20:
            severity = "HIGH" if abs(revenue_change) > 30 else "MEDIUM"
            direction = "increased" if revenue_change > 0 else "decreased"
            self.alerts.append({
                'type': 'Revenue Anomaly',
                'severity': severity,
                'message': f'Revenue has {direction} by {abs(revenue_change):.1f}% in the last week',
                'value': recent_avg_revenue,
                'change': revenue_change
            })
        
        # Conversion rate monitoring
        recent_conversion = recent_7_days['conversion_rate'].mean()
        previous_conversion = previous_7_days['conversion_rate'].mean()
        conversion_change = ((recent_conversion - previous_conversion) / previous_conversion) * 100
        
        if conversion_change < -10:
            self.alerts.append({
                'type': 'Conversion Drop',
                'severity': 'HIGH',
                'message': f'Conversion rate dropped by {abs(conversion_change):.1f}% - investigate marketing channels',
                'value': recent_conversion,
                'change': conversion_change
            })
        
        # Return rate spike detection
        recent_returns = recent_7_days['return_rate'].mean()
        if recent_returns > 0.08:  # 8% threshold
            self.alerts.append({
                'type': 'High Return Rate',
                'severity': 'MEDIUM',
                'message': f'Return rate is {recent_returns:.1%} - check product quality',
                'value': recent_returns,
                'change': 0
            })
        
        return self.alerts
    
    def generate_insights(self):
        """Generate actionable business insights"""
        
        # Seasonal analysis
        monthly_data = self.data.groupby('month')['revenue'].mean()
        peak_month = monthly_data.idxmax()
        low_month = monthly_data.idxmin()
        
        self.insights.append({
            'category': 'Seasonality',
            'insight': f'Peak sales occur in month {peak_month}, lowest in month {low_month}',
            'action': 'Plan inventory and marketing campaigns accordingly',
            'impact': 'HIGH'
        })
        
        # Customer behavior patterns
        weekend_revenue = self.data[self.data['is_weekend']]['revenue'].mean()
        weekday_revenue = self.data[~self.data['is_weekend']]['revenue'].mean()
        
        if weekend_revenue < weekday_revenue * 0.8:
            self.insights.append({
                'category': 'Customer Behavior',
                'insight': f'Weekend revenue is {(weekend_revenue/weekday_revenue):.1%} of weekday revenue',
                'action': 'Consider weekend-specific promotions or marketing',
                'impact': 'MEDIUM'
            })
        
        # Marketing efficiency
        recent_cac = self.data.tail(30)['customer_acquisition_cost'].mean()
        recent_aov = self.data.tail(30)['avg_order_value'].mean()
        
        if recent_cac > recent_aov * 0.3:
            self.insights.append({
                'category': 'Marketing Efficiency',
                'insight': f'Customer acquisition cost (${recent_cac:.0f}) is high relative to order value',
                'action': 'Optimize marketing channels and improve conversion rates',
                'impact': 'HIGH'
            })
        
        return self.insights
    
    def create_executive_summary(self):
        """Create automated executive summary"""
        recent_30 = self.data.tail(30)
        previous_30 = self.data.iloc[-60:-30]
        
        summary = {
            'period': f"{recent_30['date'].min().strftime('%Y-%m-%d')} to {recent_30['date'].max().strftime('%Y-%m-%d')}",
            'total_revenue': recent_30['revenue'].sum(),
            'total_customers': recent_30['customers'].sum(),
            'avg_order_value': recent_30['avg_order_value'].mean(),
            'conversion_rate': recent_30['conversion_rate'].mean(),
            'revenue_growth': ((recent_30['revenue'].sum() - previous_30['revenue'].sum()) / previous_30['revenue'].sum()) * 100,
            'customer_growth': ((recent_30['customers'].sum() - previous_30['customers'].sum()) / previous_30['customers'].sum()) * 100
        }
        
        return summary

# Initialize BI Alert System
bi_alert = BusinessIntelligenceAlert(df_business)

# Generate alerts
alerts = bi_alert.detect_anomalies()
insights = bi_alert.generate_insights()
executive_summary = bi_alert.create_executive_summary()

print("🚨 BUSINESS ALERTS")
print("-" * 25)
if alerts:
    for alert in alerts:
        severity_icon = "🔴" if alert['severity'] == 'HIGH' else "🟡" if alert['severity'] == 'MEDIUM' else "🟢"
        print(f"{severity_icon} {alert['type']} ({alert['severity']})")
        print(f"   {alert['message']}")
        print()
else:
    print("✅ No critical alerts detected")

print("\n💡 BUSINESS INSIGHTS")
print("-" * 25)
for insight in insights:
    impact_icon = "🔥" if insight['impact'] == 'HIGH' else "⚡" if insight['impact'] == 'MEDIUM' else "💫"
    print(f"{impact_icon} {insight['category']} ({insight['impact']} Impact)")
    print(f"   📊 Insight: {insight['insight']}")
    print(f"   🎯 Action: {insight['action']}")
    print()

print("\n📋 EXECUTIVE SUMMARY")
print("-" * 25)
print(f"Period: {executive_summary['period']}")
print(f"Total Revenue: ${executive_summary['total_revenue']:,.0f} ({executive_summary['revenue_growth']:+.1f}%)")
print(f"Total Customers: {executive_summary['total_customers']:,.0f} ({executive_summary['customer_growth']:+.1f}%)")
print(f"Average Order Value: ${executive_summary['avg_order_value']:,.0f}")
print(f"Conversion Rate: {executive_summary['conversion_rate']:.1%}")

# Create Alert Dashboard
print("\n📊 Creating Alert Dashboard...")

fig_alerts = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Revenue Trend with Alerts', 'Conversion Rate Monitoring', 
                   'Return Rate Tracking', 'Customer Acquisition Cost'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Recent data for detailed monitoring
recent_60_days = df_business.tail(60)

# 1. Revenue with alert zones
fig_alerts.add_trace(
    go.Scatter(x=recent_60_days['date'], y=recent_60_days['revenue'],
              mode='lines+markers', name='Daily Revenue',
              line=dict(color='#1f77b4', width=2)),
    row=1, col=1
)

# Add alert threshold lines
avg_revenue = recent_60_days['revenue'].mean()
upper_threshold = avg_revenue * 1.2
lower_threshold = avg_revenue * 0.8

fig_alerts.add_hline(y=upper_threshold, line_dash="dash", line_color="green", 
                    annotation_text="Upper Alert Threshold", row=1, col=1)
fig_alerts.add_hline(y=lower_threshold, line_dash="dash", line_color="red", 
                    annotation_text="Lower Alert Threshold", row=1, col=1)

# 2. Conversion Rate
fig_alerts.add_trace(
    go.Scatter(x=recent_60_days['date'], y=recent_60_days['conversion_rate']*100,
              mode='lines+markers', name='Conversion Rate %',
              line=dict(color='#ff7f0e', width=2)),
    row=1, col=2
)

# 3. Return Rate
fig_alerts.add_trace(
    go.Scatter(x=recent_60_days['date'], y=recent_60_days['return_rate']*100,
              mode='lines+markers', name='Return Rate %',
              line=dict(color='#d62728', width=2)),
    row=2, col=1
)

fig_alerts.add_hline(y=8, line_dash="dash", line_color="red", 
                    annotation_text="Alert Threshold (8%)", row=2, col=1)

# 4. Customer Acquisition Cost
fig_alerts.add_trace(
    go.Scatter(x=recent_60_days['date'], y=recent_60_days['customer_acquisition_cost'],
              mode='lines+markers', name='CAC ($)',
              line=dict(color='#2ca02c', width=2)),
    row=2, col=2
)

# Update layout
fig_alerts.update_layout(
    height=800,
    showlegend=False,
    title_text="<b>Business Intelligence Alert Dashboard</b>",
    title_x=0.5
)

fig_alerts.show()

print("✅ Automated BI system deployed!")
print("\n🔧 System Features:")
print("  • Real-time anomaly detection")
print("  • Automated insight generation")
print("  • Executive summary creation")
print("  • Visual alert dashboards")
print("  • Threshold-based monitoring")

🤖 AUTOMATED BUSINESS INSIGHTS AND ALERTS
🚨 BUSINESS ALERTS
-------------------------
✅ No critical alerts detected

💡 BUSINESS INSIGHTS
-------------------------
🔥 Seasonality (HIGH Impact)
   📊 Insight: Peak sales occur in month 6, lowest in month 1
   🎯 Action: Plan inventory and marketing campaigns accordingly

⚡ Customer Behavior (MEDIUM Impact)
   📊 Insight: Weekend revenue is 70.9% of weekday revenue
   🎯 Action: Consider weekend-specific promotions or marketing

🔥 Marketing Efficiency (HIGH Impact)
   📊 Insight: Customer acquisition cost ($51) is high relative to order value
   🎯 Action: Optimize marketing channels and improve conversion rates


📋 EXECUTIVE SUMMARY
-------------------------
Period: 2025-06-20 to 2025-07-19
Total Revenue: $3,577,081 (-3.6%)
Total Customers: 18,527 (-13.8%)
Average Order Value: $147
Conversion Rate: 24.3%

📊 Creating Alert Dashboard...


✅ Automated BI system deployed!

🔧 System Features:
  • Real-time anomaly detection
  • Automated insight generation
  • Executive summary creation
  • Visual alert dashboards
  • Threshold-based monitoring
